<a href="https://colab.research.google.com/github/BuddhiGamage/Sinhala-Word-Embeddings-Trainer/blob/feature%2Ftrain-embedding/Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Word Embedding 

In [ ]:
#Start by connecting gdrive into the google colab
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


## Pre Proccessing

In [ ]:
## Masking

## Train model

imports

In [ ]:
import os
import sys
from os import listdir
from os.path import isfile, join
import nltk
nltk.download('punkt')

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

#from glove import Corpus, Glove

from pathlib import Path

import time
from datetime import timedelta


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Iterate through txt files in the given path . 
Read the files and return the training data

In [ ]:
def get_training_data(rootdir):
   """
  Iterate through folders sub folders
  rootdir: directory where files locate
  
  Read the files in each path line by line.
  Each sentence is tokenized
  Returns:
    sentence_list = list of tokenized sentences.

  """

  file_list=[]
  for root, dirs, files in os.walk(rootdir):
        for name in files:
            file_list.append(os.path.join(root, name))

   
  sentence_list = []
  tokenize  = lambda x: nltk.word_tokenize(x)


  for file_name in file_list:
    with open(file_name, 'r', encoding="utf-8") as text_file:
      text = text_file.readlines()
      for line in text:
        #Preprocess function
        #line = preprocess_text(line)
        sentence_list.append(tokenize(line))

  print('Found %s texts.' % len(sentence_list))

  return sentence_list


In [ ]:
list2= get_training_data('/content/gdrive/MyDrive/Subasa V3.0 /Notebooks/Test Data',)

Found 37 texts.


### Train the FastText and Wordvec word embediings

In [ ]:

def train(train_data_path, iter=500, sg=0, window=10, model='fasttext'):
    """
    Train word embedding model
    Args:
        train_data_path: corpus path
        iter: number of iterations
        window: Window size
        sg: 1 for skip gram 0 for CBOW
        model: word embedding algorithms. Either 'fasttext' or 'word2vec'
    Returns:
        model: word embedding model
    """
    train_data = get_training_data(train_data_path)
    

    #if you want you can save the processed corpus into seperate file in the disk so that you can save your time next time training
    pickle_out = open("train_data.pickle","wb")
    pickle.dump(train_data, pickle_out)
    pickle_out.close()  
    
    assert model == 'fasttext' or model == 'word2vec', "model must be fasttext or word2vec"

    if model == 'fasttext':
        print('Training model', model,'.....')
        # build vocabulary and train model
        model = FastText(
            train_data,
            size=300,
            window=window,
            min_count=5,
            workers=10,
            iter=iter,
            sg=sg)
        return model

    elif model == 'word2vec':
        print('Training model', model, '.....')
        # build vocabulary and train model
        model = Word2Vec(
            train_data,
            size=300,
            window=window,
            min_count=5,
            workers=10,
            iter=iter,
            sg=sg)
        return model


def save_embedding_model(model,model_dir,model_name):
    """
    Save trained embedding model
    Args:
        model: word embedding model to save
        model_dir: word embedding model directory
        model_name: model name
    """
    model_dir = Path(model_dir)
   
    if not model_dir.exists():
        model_dir.mkdir()
    print('Saving model in ', model_dir, ' .....')
    model.save(os.path.join(model_dir,'{}.bin'.format(model_name)))
    model.wv.save_word2vec_format(os.path.join(model_dir,'{}.txt'.format(model_name)))




In [ ]:
model = train('/content/gdrive/MyDrive/Subasa V3.0 /Notebooks/Test Data', iter=20,sg=0, window=10, model='fasttext')
save_embedding_model(model,'./model2','sin')
#print ('Elapsed Time:', str(timedelta(seconds=(time.time()-start_time)))    

Training model fasttext .....


In [ ]:
model = FastText.load('/content/model2/sin.bin')
print(model)

FastText(vocab=139940, size=300, alpha=0.025)


In [ ]:
model.most_similar('රජය',topn=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('රජයද', 0.8568500280380249),
 ('රජයම', 0.8442015647888184),
 ('රජයන', 0.7830795049667358),
 ('ආණ්ඩුව', 0.7629334926605225),
 ('රජයෙ', 0.7617363333702087),
 ('රජයටද', 0.7422264218330383),
 ('රජයන්', 0.7372043132781982),
 ('රජයත්', 0.7369410991668701),
 ('රජයයන්', 0.7320923805236816),
 ('රජයේද', 0.7236485481262207)]

### Hyperparameter Tuning